In [4]:
! pip install requests  bs4  selenium  tqdm 

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached selenium-4.7.2-py3-none-any.whl (6.3 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached exceptiongroup-1.1.0-py3-none-any.whl (14 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=50303

In [1]:
import os
import re
import time
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm


In [2]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

buffer_size = 1024

In [3]:

def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_images_from_link(url):
    """
    Function to find image links in a given website
    :param url: the URL of the desired website
    :return: a list of image links (URLs)
    """

    try:
        soup = bs(requests.get(url).content, "html.parser")
    except:
        return []

    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        alt = img.attrs.get("alt")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)

        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass

            # finally, if the url is valid
            if is_valid(img_url):
                urls.append(img_url)
    return urls


def download_image_from_url(url, pathname):
    """
    Download an image online from a URL and save it locally
    :param url: the image URL
    :param pathname: where to save the image
    :return: non
    """

    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    try:
        response = requests.get(url, stream=True)
    except:
        return 0
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    if file_size < 30 * 1024:
        return 0

    try:
        img_name = url.split("/")[-1]
        img_name = img_name[:150]
        img_name = re.sub(r'[\\:/*"<>]', '_', img_name)

        # get the file name
        filename = os.path.join(pathname, img_name)

        # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
        progress = tqdm(response.iter_content(buffer_size), f"Downloading {filename}", total=file_size, unit="B",
                        unit_scale=True, unit_divisor=1024)
        with open(filename, "wb") as f:
            for data in progress:
                # write data read to the file
                f.write(data)
                # update the progress bar manually
                progress.update(len(data))
    except:
        return 0


def download_images_from_website(url, path):
    """
    Download all the images from a given URL
    :param url: the website URL
    :param path: where to save these images
    :return: none
    """
    imgs = get_all_images_from_link(url)
    imgs = list(set(imgs))
    for img in imgs:
        # for each image, download it
        if img.split('.')[-1] in ['jpg', 'png', 'jpeg', 'jfif']:
            download_image_from_url(img, path)


def search_google_images(browser, url, query, max_google_pages=1):
    """
    Search a query keywork in Google images
    :param url: the URL of google images
    :param query: the keyword to search for
    :return: the HTML code of the Google Image result page
    """

    # Open the link
    browser.get(url)
    time.sleep(1)
    
    # Accept Google cookies:
    browser.find_element(By.XPATH,"//.[@aria-label='Accept all']").click()

    print("[%] Successfully opened link.")

    element = browser.find_element(By.TAG_NAME,'body')# browser.find_element_by_name("body")

    print("[%] Scrolling down.")
    # Scroll down
    for i in range(max_google_pages):
        element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)  # bot id protection

    try:
        for i in range(max_google_pages):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)  # bot id protection
    except Exception:
        for i in range(10):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)  # bot id protection

    print("[%] Reached end of Page.")

    time.sleep(1)
    # Get page source and close the browser
    source = browser.page_source
    with open("{}/dataset/soups/{}.html".format(os.getcwd(), query), 'w', encoding='utf-8',
              errors='replace') as f:
        f.write(source)
    browser.close()
    print("[%] Closed Driver.")

    return source


In [4]:

# List of queries to search in Google images
queries = ['cute funny dogs']
max_google_pages = 1

google_url_base = "https://www.google.com/search?q={}&source=lnms&tbm=isch"

# Create a folder to save the output
cwd = os.getcwd()
if not os.path.isdir("{}/dataset/".format(cwd)):
    os.makedirs("{}/dataset/".format(cwd))
if not os.path.isdir("{}/dataset/soups/".format(cwd)):
    os.makedirs("{}/dataset/soups/".format(cwd))


# Open FireFox browser
firefox_binary = FirefoxBinary()
browser = webdriver.Firefox(firefox_binary=firefox_binary)
#     browser.set_window_size(1024, 768)
#     browser.minimize_window()
print("\n===============================================\n")
print("[%] Successfully launched FirefoxBinary")


for query in queries:
    url = google_url_base.format(query)

    # Return the HTML page of Google Image search of the query
    source_html = search_google_images(browser, url, query.replace(' ', '_'), max_google_pages=max_google_pages)

    # Parse the HTML using BS4
    soup = bs(source_html, "html.parser")

    # Find all images href
    all_href = soup.find_all('a', href=True)

    # Return the original Web pages of these images
    pages = []
    for a in all_href:
        link = a.attrs.get('href')
        if link.startswith('http'):
            pages.append(link)
    pages = list(set(pages))

    # download the images from their official website
    for p in pages:
        download_images_from_website(p, './dataset/images/{}/'.format(query))


C:\Users\GVIS\AppData\Local\Temp\ipykernel_7500\1438101816.py:17: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(firefox_binary=firefox_binary)




[%] Successfully launched FirefoxBinary
[%] Successfully opened link.
[%] Scrolling down.
[%] Reached end of Page.
[%] Closed Driver.


Extracting images: 0it [00:00, ?it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 31099.23it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 10089.13it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 23240.72it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 11608.68it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2084.64it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|██████████████████████████████████

Extracting images: 100%|████████████████████████████████████████████████████████████| 88/88 [00:00<00:00, 85045.80it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1906.94it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 2001.58it/s]
Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|████████████████████████████████████████████████████████████| 43/43 [00:00<00:00, 43023.63it/s]


Extracting images: 0it [00:00, ?it/s]
Extracting images: 100%|██████████████████████████████████████████████████████████| 397/397 [00:00<00:00, 15638.63it/s]


Extracting images: 100%|██████████████████████████████████████████████████████████| 827/827 [00:00<00:00, 11311.45it/s]


Extracting images: 100%|████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 34877.80it/s]
Extracting images: 0it [00:00, ?it/s]
